# Diet Calculator

Alternatively, you can visit http://frepha.pythonanywhere.com/ to use the web application.

In [141]:
import pandas as pd
import numpy as np
from pulp import *
import re
import matplotlib
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

In [142]:
# change the directory to your own one
#df = pd.read_excel("./NutrientsTest.xlsx")

In [143]:
veg = df['Category'] == "Vegetables"
fruit = df['Category'] == "Fruits"

df['Meat'] = df['Category'] == "Meat"
df['Meat'] = df['Meat'].astype(int)
df['FruitVeg'] = fruit | veg
df['FruitVeg'] = df['FruitVeg'].astype(int)
df['Condiment'] = df['Category'] == "Condiment"
df['Condiment'] = df['Condiment'].astype(int)

In [144]:
#select columns
columns3 = []
columns3.append("Category")
columns3.append("Food Name")
columns3.append("Price (RM)")
columns3.append("Fat, total (g/100g)")
columns3.append("Fibre, total dietary (g/100g)")
columns3.append("Protein, total; calculated from total nitrogen (g/100g)")
columns3.append("Sugars, total (g/100g)")
columns3.append("Fatty acids, total saturated (g/100g)")
columns3.append("Sugar, added")
columns3.append("Cholesterol (mg/100g)")
columns3.append("Energy (kcal/100g)")
columns3.append("Meat")
columns3.append("FruitVeg")
columns3.append("Vitamin C (mg/100g)")
columns3.append("Vitamin E, alpha-tocopherol equivalents (mg/100g)")
columns3.append("Calcium (mg/100g)")
columns3.append("Sodium (mg/100g)")
columns3.append("Iron (mg/100g)")
columns3.append("Zinc (mg/100g)")
columns3.append("Vitamin A, retinol equivalents (µg/100g)")
columns3.append("Vitamin D (μg/100g)")
columns3.append("Vitamin K (μg/100g)")
columns3.append("Manganese (μg/100g)")
columns3.append("Fat, total (g/100g)")
columns3.append("Fibre, total dietary (g/100g)")
columns3.append("Sodium (mg/100g)")
columns3.append("Condiment")
columns3

df2 = df[columns3]
data = df2.values.tolist()

In [145]:
def get_nutrient_constraints(gender):
    # Nutrient constraints for three categories.
    if (gender == 'male'):
        nutrients = [
            ["Fat, total (g/100g)", 73],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 61],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 2190],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 70],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 10],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 9],
            ["Zinc (mg/100g)", 6.5],
            ["Vitamin A, retinol equivalents (µg/100g)", 600],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 65],
            ["Manganese (μg/100g)", 2300],
            ["Fat, total (g/100g)", 61],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1500],
            ["Condiment", 0.06]
        ]
    elif (gender == 'female'):
        nutrients = [
            ["Fat, total (g/100g)", 63],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 52],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 1900],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 70],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 7.5],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 20],
            ["Zinc (mg/100g)", 4.6],
            ["Vitamin A, retinol equivalents (µg/100g)", 600],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 55],
            ["Manganese (μg/100g)", 1800],
            ["Fat, total (g/100g)", 53],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1500],
            ["Condiment", 0.06]
        ]
    elif (gender == 'children'):
        nutrients = [
            ["Fat, total (g/100g)", 68],
            ["Fibre, total dietary (g/100g)", 20],
            ["Protein, total; calculated from total nitrogen (g/100g)", 23],
            ["Sugars, total (g/100g)", 325],
            ["Fatty acids, total saturated (g/100g)", 20],
            ["Sugar, added", 50],
            ["Cholesterol (mg/100g)", 300],
            ["Energy (kcal/100g)", 1750],
            ["Meat", 1],
            ["FruitVeg", 4],
            ["Vitamin C (mg/100g)", 35],
            ["Vitamin E, alpha-tocopherol equivalents (mg/100g)", 7],
            ["Calcium (mg/100g)", 1000],
            ["Sodium (mg/100g)", 2300],
            ["Iron (mg/100g)", 6],
            ["Zinc (mg/100g)", 5.7],
            ["Vitamin A, retinol equivalents (µg/100g)", 500],
            ["Vitamin D (μg/100g)", 15], #!!!
            ["Vitamin K (μg/100g)", 25],
            ["Manganese (μg/100g)", 1500],
            ["Fat, total (g/100g)", 49],
            ["Fibre, total dietary (g/100g)", 70],
            ["Sodium (mg/100g)", 1200],
            ["Condiment", 0.06]
        ]
    return nutrients

In [146]:
def optimize_cost(gender):
    # Instantiate a Glop solver and naming it.
    solver = pywraplp.Solver('ZahirDiet', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Declare an array to hold our variables.
    # 0 lower bound, 3 upper bound (max 300 gram per food)

    foods = [solver.NumVar(0, 3, item[1]) for item in data]
    print('Number of decision variables: ', solver.NumVariables())
    
    # index of constraints for inequality (>=)
    more = [ 1, 2, 7, 9, 10, 11, 12, 14, 
            15, 16, 17, 18, 19, 20, 22 ]

    nutrients = get_nutrient_constraints(gender)
    constraints = []
    df_food = ""
    df_nutrient = ""
    #print("Constraints:\n----------------------------")

    for i, nutrient in enumerate(nutrients):
        if (i in more):
            #print('{} >= {:.2f}'.format(nutrient[0], nutrient[1]))
            constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
        else:
            #print('{} <= {:.2f}'.format(nutrient[0], nutrient[1]))
            constraints.append(solver.Constraint(-solver.infinity(), nutrient[1]))

        for j, item in enumerate(data):
            constraints[i].SetCoefficient(foods[j], item[i + 3])

    #print('----------------------------\nNumber of constraints =', solver.NumConstraints())\
    
    # Objective function: Minimize the sum of (price-normalized) foods.
    objective = solver.Objective()
    for i, food in enumerate(foods):
        objective.SetCoefficient(food, df2.loc[i]['Price (RM)'])
    objective.SetMinimization()

    status = solver.Solve()

    # Check that the problem has an optimal solution.
    if status != solver.OPTIMAL:
        print('The problem does not have an optimal solution!')
        if status == solver.FEASIBLE:
            print('A potentially suboptimal solution was found.')
        else:
            print('The solver could not solve the problem.')
            exit(1)

    #if there is optimal solution
    else:
        # Display the amounts (in dollars) to purchase of each food.
        nutrients_result = [0] * len(nutrients)
        price_table = [] #food list
        table = [] #nutrient list

        for i, food in enumerate(foods):
            if food.solution_value() > 0.0:
                price_table.append([data[i][1], food.solution_value()*100, food.solution_value()*data[i][2]])
                for j, _ in enumerate(nutrients):
                    nutrients_result[j] += data[i][j + 3] * food.solution_value()

        print('==============================\nDiet (Optimal) Price: RM{:.2f}\n=============================='.format(objective.Value()))

        for i, nutrient in enumerate(nutrients):
            if (i in more):
                table.append([nutrient[0], '≥ ' + str(nutrient[1]), nutrients_result[i]])
            else:
                table.append([nutrient[0], '≤ ' + str(nutrient[1]), nutrients_result[i]])

        column_names = ['Food', 'Gram', 'Price']
        df_food = pd.DataFrame(price_table, columns = column_names)
        df_food = df_food.sort_values(by = ['Food'], ascending=[True]).reset_index(drop=True)
        df_food = df_food.round(2)

        column_names = ['Nutrient', 'Constraint', 'Current Diet Value']
        df_nutrient = pd.DataFrame(table, columns = column_names)
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fat, total (g/100g)", ["Nutrient"]] = "Total Fat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Calcium (mg/100g)", ["Nutrient"]] = "Calcium (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Cholesterol (mg/100g)", ["Nutrient"]] = "Cholesterol (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Energy (kcal/100g)", ["Nutrient"]] = "Calories (kcal)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fatty acids, total saturated (g/100g)", ["Nutrient"]] = "Total Sat. Fat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Fibre, total dietary (g/100g)", ["Nutrient"]] = "Fibre (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "FruitVeg", ["Nutrient"]] = "Fruits & Vegetables (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Meat", ["Nutrient"]] = "Meat (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin K (μg/100g)", ["Nutrient"]] = "Vitamin K (μg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin E, alpha-tocopherol equivalents (mg/100g)", ["Nutrient"]] = "Vitamin E (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin D (μg/100g)", ["Nutrient"]] = "Vitamin D (μg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Iron (mg/100g)", ["Nutrient"]] = "Iron (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sodium (mg/100g)", ["Nutrient"]] = "Sodium (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sugar, added", ["Nutrient"]] = "Added Sugar (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Sugars, total (g/100g)", ["Nutrient"]] = "Carbohydrates (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin A, retinol equivalents (µg/100g)", ["Nutrient"]] = "Vitamin A (µg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Vitamin C (mg/100g)", ["Nutrient"]] = "Vitamin C (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Manganese (μg/100g)", ["Nutrient"]] = "Manganese (µg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Zinc (mg/100g)", ["Nutrient"]] = "Zinc (mg)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Protein, total; calculated from total nitrogen (g/100g)", ["Nutrient"]] = "Protein (g)"
        df_nutrient.loc[df_nutrient["Nutrient"] == "Condiment", ["Nutrient"]] = "Condiment (g)"
        df_nutrient = df_nutrient.sort_values(by = ['Nutrient', 'Constraint'], ascending=[True, False]).reset_index(drop=True)
        df_nutrient = df_nutrient.round(2)
    
    return df_food, df_nutrient

<hr>

### Diet for Male Adult
#### Diet Price

In [147]:
df_food, df_nutrient = optimize_cost('male')

Number of decision variables:  150
Diet (Optimal) Price: RM15.59


#### Diet Plan

In [148]:
df_food

,Food,Gram,Price
0,"Egg, chicken, white & yolk, raw",74.49,0.48
1,"Milk, cow, chocolate flavour, fluid, ultra-hig...",115.23,0.82
2,"Nut, almond, dry roasted, unblanched, no salt ...",48.61,1.94
3,"Oatmeal, dry",300.00,3.75
4,"Pineapple, flesh, raw",300.00,1.18
5,"Rice, white, regular, assorted grains, polishe...",42.58,0.11
6,"Salt, table, iodised",3.20,0.06
7,"Spinach, leafy vegetable, raw, English",56.52,1.27
8,"Sugar, brown",2.80,0.01
9,"Watermelon, flesh, raw, combined varieties",43.48,0.17


#### Diet Constraints

In [149]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 2190,2190.00
3,Carbohydrates (g),≤ 325,60.43
4,Cholesterol (mg),≤ 300,300.00
5,Condiment (g),≤ 0.06,0.06
6,Fibre (g),≥ 20,45.39
7,Fibre (g),≤ 70,45.39
8,Fruits & Vegetables (g),≥ 4,4.00
9,Iron (mg),≥ 9,28.19


<hr>

### Diet for Female Adult
#### Diet Price

In [103]:
df_food, df_nutrient = optimize_cost('female')

Number of decision variables:  150
Diet (Optimal) Price: RM14.32


#### Diet Plan

In [104]:
df_food

,Food,Gram,Price
0,"Cabbage, leafy vegetable, raw, choy sum",68.24,0.91
1,"Carrot, root vegetable, taproot, flesh, fresh,...",6.41,0.04
2,"Egg, chicken, white & yolk, raw",74.08,0.48
3,"Milk, cow, chocolate flavour, fluid, ultra-hig...",148.02,1.05
4,"Nut, almond, dry roasted, unblanched, no salt ...",22.84,0.91
5,"Oatmeal, dry",299.19,3.74
6,"Pineapple, flesh, raw",300.00,1.18
7,"Pineapple, unripe, flesh",4.58,0.02
8,"Powder, curry",2.88,0.05
9,"Salt, table, iodised",3.12,0.06


#### Diet Constraints

In [105]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 1900,1900.00
3,Carbohydrates (g),≤ 325,58.49
4,Cholesterol (mg),≤ 300,300.00
5,Condiment (g),≤ 0.06,0.06
6,Fibre (g),≥ 20,42.76
7,Fibre (g),≤ 70,42.76
8,Fruits & Vegetables (g),≥ 4,4.00
9,Iron (mg),≥ 20,27.06


<hr>

### Diet for Children
#### Diet Price

In [106]:
df_food, df_nutrient = optimize_cost('children')

Number of decision variables:  150
Diet (Optimal) Price: RM13.62


#### Diet Plan

In [107]:
df_food

,Food,Gram,Price
0,"Cabbage, leafy vegetable, raw, choy sum",31.02,0.42
1,"Chicken, liver, raw",0.33,0.00
2,"Egg, chicken, white & yolk, raw",72.79,0.47
3,"Milk, cow, chocolate flavour, fluid, ultra-hig...",224.27,1.59
4,"Nut, almond, dry roasted, unblanched, no salt ...",16.21,0.65
5,"Oatmeal, dry",255.32,3.19
6,"Pineapple, flesh, raw",260.97,1.03
7,"Pineapple, unripe, flesh",108.01,0.37
8,"Powder, curry",3.71,0.07
9,"Salt, table, iodised",2.29,0.05


#### Diet Constraints

In [108]:
df_nutrient

,Nutrient,Constraint,Current Diet Value
0,Added Sugar (g),≤ 50,50.00
1,Calcium (mg),≥ 1000,1000.00
2,Calories (kcal),≥ 1750,1750.00
3,Carbohydrates (g),≤ 325,67.08
4,Cholesterol (mg),≤ 300,300.00
5,Condiment (g),≤ 0.06,0.06
6,Fibre (g),≥ 20,37.23
7,Fibre (g),≤ 70,37.23
8,Fruits & Vegetables (g),≥ 4,4.00
9,Iron (mg),≥ 6,24.24


In [109]:
data

[['Fruits',
  "Apple, flesh & skin, raw, 'Granny Smith' (Green Apple)",
  1.4875,
  0.3,
  2.41,
  0.34,
  8.5,
  0.016,
  0.0,
  0.0,
  44.43,
  0,
  1,
  4.49,
  0.28,
  3.9,
  0.0,
  0.0,
  0.0,
  3.46,
  0.0,
  0.0,
  34.0,
  0.3,
  2.41,
  0.0,
  0],
 ['Fruits',
  'Apple, flesh & skin, raw, Red Delicious',
  1.4222222222222223,
  0.49,
  2.0,
  0.26,
  10.5,
  0.128,
  0.0,
  0.0,
  56.25,
  0,
  1,
  4.5,
  0.33,
  4.68,
  0.85,
  0.14,
  0.03,
  7.52,
  0.0,
  0.0,
  21.2,
  0.49,
  2.0,
  0.85,
  0],
 ['Fruits',
  "Apple, flesh & skin, raw, 'Royal Gala'",
  1.3,
  0.3,
  1.98,
  0.24,
  10.5,
  0.016,
  0.0,
  0.0,
  51.33,
  0,
  1,
  2.85,
  1.07,
  6.3,
  1.0,
  0.12,
  0.04,
  3.46,
  0.0,
  0.0,
  33.0,
  0.3,
  1.98,
  1.0,
  0],
 ['Fruits',
  'Banana, flesh & seed, raw, ripened, yellow, composite',
  0.5,
  0.3,
  1.8,
  1.03,
  15.2,
  0.096,
  3.38,
  0.0,
  94.86,
  0,
  1,
  5.5,
  0.2,
  5.3,
  0.0,
  0.23,
  0.17,
  6.08,
  0.0,
  0.5,
  380.0,
  0.3,
  1.8,
  0.0,

In [113]:
df_price = df['Price (RM)']
df_price.to_csv("Price.csv", index=False)